# JON'S HiML Competition RFG_Log Model_picker (v 2.RFG)
## Make Date: 04/13/18

1. This ML model was built using scikit's RandomForestRegressor model.  
1. Automatically find the best combination of input parameters using the combo-picker function    
1. Let's implement log transform on the quantities to improve score.



In [2]:
#Some initialization procedures:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# load in data files
# FILE_DIR = '../input/hawaiiml-data'
# for f in os.listdir(FILE_DIR):
#     print('{0:<30}{1:0.2f}MB'.format(f, 1e-6*os.path.getsize(f'{FILE_DIR}/{f}')))
FILE_DIR = '../Sprint09alt_Machine_Learning_Hawaii_Kaggle_Competition'
df_train = pd.read_csv(f'{FILE_DIR}/train_rm.csv', encoding='ISO-8859-1') #write training data to dataframe
df_test = pd.read_csv(f'{FILE_DIR}/test_rm.csv', encoding='ISO-8859-1') # Read the test data

#define the error function:
def rmsle(y_true, y_pred):
    return np.sqrt(np.mean((np.log1p(y_true) - np.log1p(y_pred))**2))

# Build Features

In [3]:
# #convert date strings to datatime datatypes:
# df_train['date'] = pd.to_datetime(df_train['date'])
# df_test['date'] = pd.to_datetime(df_test['date'])

# df_train = df_train.assign(day_of_week = df_train.date.dt.dayofweek)
# df_test = df_test.assign(day_of_week = df_test.date.dt.dayofweek)

In [4]:
# df_train = df_train.assign(is_day0To2 = 0)
# df_train['is_day0To2'] = 0
# df_train.loc[(df_train['day_of_week'] < 3),['is_day0To2']]  = 1

# df_train = df_train.assign(is_day3 = 0)
# df_train['is_day3'] = 0
# df_train.loc[(df_train['day_of_week'] == 3),['is_day3']]  = 1

# df_train = df_train.assign(is_day4 = 0)
# df_train['is_day4'] = 0
# df_train.loc[(df_train['day_of_week'] == 4),['is_day4']]  = 1

# df_train = df_train.assign(is_day6 = 0)
# df_train['is_day6'] = 0
# df_train.loc[(df_train['day_of_week'] == 6),['is_day6']]  = 1

In [5]:
df_train = df_train.loc[(df_train['unit_price'] > 0) & (df_train['unit_price'] < 10000) ]
df_train = df_train.assign(unit_price_log1p = np.log1p(df_train.unit_price))
df_test = df_test.assign(unit_price_log1p = np.log1p(df_test.unit_price))

In [6]:
# df_train = df_train.assign(uplog1p_quant1 = 0)
# df_train.loc[(df_train['unit_price_log1p'] < 6.47774129795256),['uplog1p_quant1']]  = 1
# df_test = df_test.assign(uplog1p_quant1 = 0)
# df_test.loc[(df_test['unit_price_log1p'] < 6.47774129795256),['uplog1p_quant1']]  = 1

In [7]:
# df_train['time'] = pd.to_datetime(df_train['time'])
# df_test['time'] = pd.to_datetime(df_test['time'])
# df_train = df_train.assign(hr = df_train.time.dt.hour)
# df_test = df_test.assign(hr = df_test.time.dt.hour)

In [8]:
# df_train = df_train.assign(invoice_id_log1p = np.log1p(df_train.invoice_id))
# df_test = df_test.assign(invoice_id_log1p = np.log1p(df_test.invoice_id))

# Define Training Target Data:
1. We want to predict the quantity data field.    
1. By convention, we define this target as 'y'.  
1. From model 2.02, we know that doing a log transform on the tartget reduces skewness and improves score.  

In [9]:
y = df_train.quantity
logy = np.log1p(df_train.quantity) #take log of quantities

# Define ML Predictors
build a function to go through each combination of predictors

### Here is the list of columns we can choose predictors from. 

In [10]:
print('Column Names & Data Types: \n', df_train.dtypes)

Column Names & Data Types: 
 id                    int64
date                 object
time                 object
invoice_id            int64
stock_id              int64
customer_id           int64
country              object
description          object
unit_price          float64
quantity              int64
UK                    int64
log_quantity        float64
date2                object
time2                object
hour                  int64
desc_set              int64
desc_bag              int64
desc_red              int64
desc_heart            int64
desc_retrospot        int64
desc_vintage          int64
desc_design           int64
desc_pink             int64
desc_christmas        int64
desc_box              int64
morning               int64
afternoon             int64
evening               int64
mean_price          float64
max_price           float64
min_price           float64
median_price        float64
median_diff         float64
mean_diff           float64
min_diff           

## Define feature combination picker

In [11]:
ls_AllPredictors = ['invoice_id', 'stock_id', 'customer_id', 'unit_price_log1p', 
    'UK',
    'desc_set','desc_bag','desc_red','desc_heart','desc_retrospot','desc_vintage',
    'desc_design','desc_pink','desc_christmas','desc_box',
    'morning','afternoon','evening',
    'median_diff','mean_diff','min_diff','max_diff'
                   ]

# https://stackoverflow.com/questions/464864/how-to-get-all-possible-combinations-of-a-list-s-elements
from itertools import chain, combinations
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)  # allows duplicate elements
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))
#build combos:
ls_PredictorCombos = [list(combo[1]) for combo in enumerate(powerset(ls_AllPredictors), 1)]
#Define combo getting function:
def GetX(comboID, adf_train):
    #get X values in adf_train for the given list of predictors
    return adf_train[ls_PredictorCombos[comboID]]

# Implement Random Forest Regressor Model on Each Predictor Combo
## Define Some Helper Functions

In [12]:
def Run_RFG(SeedVal, X, logy):
    #fit, predict, and then evaluate a model using passed in values (e.g. training set)
    #return back RMSLE value and trained model

    myModel = RandomForestRegressor(n_estimators=20, n_jobs = -1)
    train_X, val_X, train_y, val_y = train_test_split(X, logy,random_state = SeedVal) #split training data into a test and train part
    myModel.fit(train_X, train_y)
    predicted_vals = np.expm1(myModel.predict(val_X)) #transform predicted values from log to "normal" Y value
    return rmsle(np.expm1(val_y), predicted_vals), myModel #include transform of val_Y values from log to "normal" Y value

In [ ]:
def FindBestCombo_Model(adf_train, logy):
    #iteratively search for best parameter combinations to match X values in adf_train with logy
    #return best combo, error results of each model, training error, and model
    MinErr = 100000000
    df_Track = pd.DataFrame()
    for comboID in range(1,len(ls_PredictorCombos)):
        print('Eval combo:', ls_PredictorCombos[comboID])
        SeedVal = 0
        X = GetX(comboID, adf_train)
        TrainErr,myModel = Run_RFG(SeedVal,X,logy)
        df2 = pd.DataFrame([[comboID, TrainErr, ','.join(ls_PredictorCombos[comboID])]],columns=['ComboID','err','Preds'])
        if df_Track.shape[0] >0:
            df_Track = pd.concat([df2, df_Track])
        else:
            df_Track = df2.copy(deep = True)
        if TrainErr < MinErr:
            MinErr = TrainErr
    #         BestSeedVal = SeedVal
            bestComboID = comboID
            print ('Best Combo: ', comboID, ' Params: ', ls_PredictorCombos[comboID], ' Err: ', TrainErr)
    #train with best combo:
    X = GetX(bestComboID, adf_train)
    TrainErr,myModel = Run_RFG(0,X,logy)
    print ('fin')
    return ls_PredictorCombos[bestComboID], df_Track, TrainErr, myModel

### Build Customer 7 and Everyone Else Models:
1. Find Best Predictors
1. Define Separate Models

In [ ]:
ls_mypredictors, df_Track, TrainErr, myModel = FindBestCombo_Model(df_train, logy)

Eval combo: ['invoice_id']
Best Combo:  1  Params:  ['invoice_id']  Err:  0.68202562531009
Eval combo: ['stock_id']
Eval combo: ['customer_id']
Eval combo: ['unit_price_log1p']
Eval combo: ['UK']
Eval combo: ['desc_set']
Eval combo: ['desc_bag']
Eval combo: ['desc_red']
Eval combo: ['desc_heart']
Eval combo: ['desc_retrospot']
Eval combo: ['desc_vintage']
Eval combo: ['desc_design']
Eval combo: ['desc_pink']
Eval combo: ['desc_christmas']
Eval combo: ['desc_box']
Eval combo: ['morning']
Eval combo: ['afternoon']
Eval combo: ['evening']
Eval combo: ['median_diff']
Eval combo: ['mean_diff']
Eval combo: ['min_diff']
Eval combo: ['max_diff']
Eval combo: ['invoice_id', 'stock_id']
Eval combo: ['invoice_id', 'customer_id']
Best Combo:  24  Params:  ['invoice_id', 'customer_id']  Err:  0.6744793903504015
Eval combo: ['invoice_id', 'unit_price_log1p']
Eval combo: ['invoice_id', 'UK']
Eval combo: ['invoice_id', 'desc_set']
Eval combo: ['invoice_id', 'desc_bag']
Eval combo: ['invoice_id', 'desc_

Eval combo: ['desc_design', 'min_diff']
Eval combo: ['desc_design', 'max_diff']
Eval combo: ['desc_pink', 'desc_christmas']
Eval combo: ['desc_pink', 'desc_box']
Eval combo: ['desc_pink', 'morning']
Eval combo: ['desc_pink', 'afternoon']
Eval combo: ['desc_pink', 'evening']
Eval combo: ['desc_pink', 'median_diff']
Eval combo: ['desc_pink', 'mean_diff']
Eval combo: ['desc_pink', 'min_diff']
Eval combo: ['desc_pink', 'max_diff']
Eval combo: ['desc_christmas', 'desc_box']
Eval combo: ['desc_christmas', 'morning']
Eval combo: ['desc_christmas', 'afternoon']
Eval combo: ['desc_christmas', 'evening']
Eval combo: ['desc_christmas', 'median_diff']
Eval combo: ['desc_christmas', 'mean_diff']
Eval combo: ['desc_christmas', 'min_diff']
Eval combo: ['desc_christmas', 'max_diff']
Eval combo: ['desc_box', 'morning']
Eval combo: ['desc_box', 'afternoon']
Eval combo: ['desc_box', 'evening']
Eval combo: ['desc_box', 'median_diff']
Eval combo: ['desc_box', 'mean_diff']
Eval combo: ['desc_box', 'min_diff

Eval combo: ['invoice_id', 'desc_heart', 'desc_vintage']
Eval combo: ['invoice_id', 'desc_heart', 'desc_design']
Eval combo: ['invoice_id', 'desc_heart', 'desc_pink']
Eval combo: ['invoice_id', 'desc_heart', 'desc_christmas']
Eval combo: ['invoice_id', 'desc_heart', 'desc_box']
Eval combo: ['invoice_id', 'desc_heart', 'morning']
Eval combo: ['invoice_id', 'desc_heart', 'afternoon']
Eval combo: ['invoice_id', 'desc_heart', 'evening']
Eval combo: ['invoice_id', 'desc_heart', 'median_diff']
Eval combo: ['invoice_id', 'desc_heart', 'mean_diff']
Eval combo: ['invoice_id', 'desc_heart', 'min_diff']
Eval combo: ['invoice_id', 'desc_heart', 'max_diff']
Eval combo: ['invoice_id', 'desc_retrospot', 'desc_vintage']
Eval combo: ['invoice_id', 'desc_retrospot', 'desc_design']
Eval combo: ['invoice_id', 'desc_retrospot', 'desc_pink']
Eval combo: ['invoice_id', 'desc_retrospot', 'desc_christmas']
Eval combo: ['invoice_id', 'desc_retrospot', 'desc_box']
Eval combo: ['invoice_id', 'desc_retrospot', 'mo

Eval combo: ['stock_id', 'desc_set', 'afternoon']
Eval combo: ['stock_id', 'desc_set', 'evening']
Eval combo: ['stock_id', 'desc_set', 'median_diff']
Eval combo: ['stock_id', 'desc_set', 'mean_diff']
Eval combo: ['stock_id', 'desc_set', 'min_diff']
Eval combo: ['stock_id', 'desc_set', 'max_diff']
Eval combo: ['stock_id', 'desc_bag', 'desc_red']
Eval combo: ['stock_id', 'desc_bag', 'desc_heart']
Eval combo: ['stock_id', 'desc_bag', 'desc_retrospot']
Eval combo: ['stock_id', 'desc_bag', 'desc_vintage']
Eval combo: ['stock_id', 'desc_bag', 'desc_design']
Eval combo: ['stock_id', 'desc_bag', 'desc_pink']
Eval combo: ['stock_id', 'desc_bag', 'desc_christmas']
Eval combo: ['stock_id', 'desc_bag', 'desc_box']
Eval combo: ['stock_id', 'desc_bag', 'morning']
Eval combo: ['stock_id', 'desc_bag', 'afternoon']
Eval combo: ['stock_id', 'desc_bag', 'evening']
Eval combo: ['stock_id', 'desc_bag', 'median_diff']
Eval combo: ['stock_id', 'desc_bag', 'mean_diff']
Eval combo: ['stock_id', 'desc_bag', 'mi

## Observations

In [ ]:
display(df_Track.sort_values(by=['err']))

In [162]:
print (myModel_7.feature_importances_)
print (myModel_no7.feature_importances_)

[ 0.64557884  0.34338557  0.01103559]
[ 0.62476342  0.37523658  0.        ]
